In [ ]:
# Import essential libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import itertools
import keras
import seaborn as sns
from glob import glob
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers.normalization import BatchNormalization

# For Resnet
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.applications import ResNet50
from keras import regularizers

# For snapshot Ensemble
from keras.callbacks import Callback
from keras import backend
from keras.models import load_model


In [ ]:
# this callback applies cosine annealing, saves snapshots and allows to load them
class SnapshotEnsemble(Callback):
    
    __snapshot_name_fmt = "snapshot_%d.hdf5"
    
    def __init__(self, n_models, n_epochs_per_model, lr_max, verbose=1):
        """
        n_models -- quantity of models (snapshots)
        n_epochs_per_model -- quantity of epoch for every model (snapshot)
        lr_max -- maximum learning rate (snapshot starter)
        """
        self.n_epochs_per_model = n_epochs_per_model
        self.n_models = n_models
        self.n_epochs_total = self.n_models * self.n_epochs_per_model
        self.lr_max = lr_max
        self.verbose = verbose
        self.lrs = []
 
    # calculate learning rate for epoch
    def cosine_annealing(self, epoch):
        cos_inner = (math.pi * (epoch % self.n_epochs_per_model)) / self.n_epochs_per_model
        return self.lr_max / 2 * (math.cos(cos_inner) + 1)

    # when epoch begins update learning rate
    def on_epoch_begin(self, epoch, logs={}):
        # update learning rate
        lr = self.cosine_annealing(epoch)
        backend.set_value(self.model.optimizer.lr, lr)
        # log value
        self.lrs.append(lr)

    # when epoch ends check if there is a need to save a snapshot
    def on_epoch_end(self, epoch, logs={}):
        if (epoch + 1) % self.n_epochs_per_model == 0:
            # save model to file
            filename = self.__snapshot_name_fmt % ((epoch + 1) // self.n_epochs_per_model)
            self.model.save(filename)
            if self.verbose:
                print('Epoch %d: snapshot saved to %s' % (epoch, filename))
                
    # load all snapshots after training
    def load_ensemble(self):
        models = []
        for i in range(self.n_models):
            models.append(load_model(self.__snapshot_name_fmt % (i + 1)))
        return models

In [ ]:
skin_directory = os.path.join('..', 'input/skin-cancer-mnist-ham10000')

# creating a directory for all images present with us and bringing them under same directory
image_directory = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(skin_directory, '*', '*.jpg'))}
# creating a directory to display the type of cancer with label values present in dataset
cancer_type_dict = {
    'nv': 'melanocytic_nevi',
    'mel': 'melanoma',
    'vasc': 'vascular_lesions',
    'df': 'dermatofibroma',
    'bkl': 'benign_keratosis_like_lesions ',
    'bcc': 'basal_cell_carcinoma',
    'akiec': 'actinic_keratoses'
}

In [ ]:

# creating data frame
skin_df = pd.read_csv(os.path.join(skin_directory, 'HAM10000_metadata.csv'))

# Creating New Columns for better readability

skin_df['path'] = skin_df['image_id'].map(image_directory.get)
skin_df['cell_type'] = skin_df['dx'].map(cancer_type_dict.get) 
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes


In [ ]:
# Now lets see the sample of tile_df to look on newly made columns
skin_df.head()

In [ ]:
# Checking for null values
skin_df.isnull().sum()

In [ ]:
# Filling null values with mean
skin_df['age'].fillna((skin_df['age'].mean()), inplace=True)

In [ ]:
skin_df.isnull().sum()

In [ ]:
print(skin_df.dtypes)

This Section Contains Data Analysis Part

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize= (7, 7))
skin_df['cell_type'].value_counts().plot(kind='pie', ax=ax1)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize= (7, 7))
skin_df['dx_type'].value_counts().plot(kind='pie',ax=ax1)

In [ ]:
skin_df['localization'].value_counts().plot(kind='bar')

In [ ]:
skin_df['age'].hist()

In [ ]:
skin_df['sex'].value_counts().plot(kind='bar')

In [ ]:
sns.scatterplot('age','cell_type_idx',data=skin_df)

In [ ]:
sns.catplot( 'sex','cell_type_idx',data=skin_df)
###sns.scatterplot('sex','cell_type_idx',data=skin_df)

In [ ]:
# Resizing images to 75 x 100
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((100,75))))

In [ ]:
# Checkin whether image column is added to skin_df or not
skin_df.head()

In [ ]:
# Printing Sample images for each lesion type
n_samples = 5
fig, m_axs = plt.subplots(7, n_samples, figsize = (4*n_samples, 3*7))
for n_axs, (type_name, type_rows) in zip(m_axs, 
                                         skin_df.sort_values(['cell_type']).groupby('cell_type')):
    n_axs[0].set_title(type_name)
    for c_ax, (_, c_row) in zip(n_axs, type_rows.sample(n_samples, random_state=1234).iterrows()):
        c_ax.imshow(c_row['image'])
        c_ax.axis('off')
fig.savefig('category_samples.png', dpi=300)

In [ ]:
# Checking the image size distribution
skin_df['image'].map(lambda x: x.shape).value_counts()

In [ ]:
df = skin_df

In [ ]:
# Print no. of instances of each class present to balace the data
skin_df['cell_type'].value_counts()

In [ ]:
# Copy fewer class to balance the number of 7 classes
data_aug_rate = [15,10,5,50,0,5,40] # These values are calculated to balance the data
for i in range(7):
    if data_aug_rate[i]:
        skin_df=skin_df.append([skin_df.loc[skin_df['cell_type_idx'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)
skin_df['cell_type'].value_counts()

This section contains data splitting and normalization

In [ ]:
features=skin_df.drop(columns=['cell_type_idx'],axis=1)
target=skin_df['cell_type_idx']

In [ ]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20,random_state=3)

In [ ]:
# Converting image data from matrix to list
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

# Normalizing the data
x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [ ]:
# Perform one-hot encoding on the labels
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)

In [ ]:
# Splitting training data and validation data
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.20, random_state = 3)

In [ ]:
# Reshape image in 3 dimensions (height = 75px, width = 100px , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [ ]:
# Model Building

model = Sequential()
num_labels = 7

base_model = ResNet50(include_top=False, input_shape=(75, 100, 3),pooling = 'avg', weights = 'imagenet')
model = Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu",kernel_regularizer=regularizers.l2(0.02)))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation = 'softmax',kernel_regularizer=regularizers.l2(0.02)))

for layer in base_model.layers:
    layer.trainable = False

for layer in base_model.layers[-22:]:
    layer.trainable = True

#model.add(ResNet50(include_top = False, pooling = 'max', weights = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'))

model.summary()

In [ ]:
# Define the optimizer
optimizer = Adam(lr=0.001)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
# With data augmentation to prevent overfitting 

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images





In [ ]:
# Fit the model

import math
print(len(x_validate))
print(len(y_validate))
se_callback = SnapshotEnsemble(n_models=3, n_epochs_per_model=10
, lr_max=.001)

history = model.fit_generator(
    datagen.flow(x_train, y_train, batch_size=10),
    steps_per_epoch=len(x_train) / 10,
    epochs=se_callback.n_epochs_total,
    verbose=1,
    callbacks=[se_callback],
    validation_data=(x_validate, y_validate)
)


Creating test data for evaluation

In [ ]:
features=df.drop(columns=['cell_type_idx'],axis=1)
target=df['cell_type_idx']

In [ ]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20,random_state=3)

In [ ]:
x_train = np.asarray(x_train_o['image'].tolist())
x_test = np.asarray(x_test_o['image'].tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [ ]:
# Perform one-hot encoding on the labels
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)

In [ ]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.20, random_state = 3)

In [ ]:
# Reshape image in 3 dimensions (height = 75px, width = 100px , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [ ]:
# Evaluating model accuracy and loss

loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model.h5")


In [ ]:
# plotting accuracy and loss of training data and validation data with progression in epochs during training process
model_history = history
fig, plots = plt.subplots(1,2,figsize=(15,5))
plots[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
plots[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
plots[0].set_title('Model Accuracy')
plots[0].set_ylabel('Accuracy')
plots[0].set_xlabel('Epoch')
plots[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
plots[0].legend(['training data', 'validation data'], loc='upper left')
plots[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
plots[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
plots[1].set_title('Model Loss')
plots[1].set_ylabel('Loss')
plots[1].set_xlabel('Epoch')
plots[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
plots[1].legend(['training data', 'validation data'], loc='upper right')
plt.show()

In [ ]:
# Predicted values
Y_pred = model.predict(x_validate)
Y_pred_test = model.predict(x_test)
# Converting to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1)
Y_pred_test_classes = np.argmax(Y_pred_test,axis = 1)
Y_true = np.argmax(y_test,axis = 1) 
# confusion matrix calculation
confusion_mtx = confusion_matrix(Y_true, Y_pred_test_classes)


#printing confusion matrix
print('------------CONFUSION MATRIX------------')
print(confusion_mtx)
print('       predicted values     ')

In [ ]:
from sklearn.metrics import roc_curve, auc

y_pred_test = model.predict(x_test)
y_pred_val = model.predict(x_validate)
num_classes=7

fpr_val = dict()
tpr_val = dict()
roc_auc_val = dict()
fpr_test = dict()
tpr_test= dict()
roc_auc_test = dict()
for i in range(num_classes):
    fpr_test[i], tpr_test[i], _ = roc_curve(y_test[:, i], y_pred_test[:, i])
    roc_auc_test[i] = auc(fpr_test[i], tpr_test[i])
    fpr_val[i], tpr_val[i], _ = roc_curve(y_validate[:, i], y_pred_val[:, i])
    roc_auc_val[i] = auc(fpr_val[i], tpr_val[i])

# Plot of a ROC curve for a specific class
for i in range(num_classes):
    plt.figure()
    plt.plot(fpr_test[i], tpr_test[i], label='ROC curve (area = %0.2f) (test dataset)' % roc_auc_test[i])
    plt.plot(fpr_val[i], tpr_val[i], label='ROC curve (area = %0.2f) (validation dataset)' % roc_auc_val[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('1-Specificity(FPR)')
    plt.ylabel('Sensitivity(TPR)')
    plt.title('Receiver operating characteristic of class: '+ str(i) + ' using test and validation datasets' )
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
correctly_predicted_frac = np.diag(confusion_mtx) / np.sum(confusion_mtx, axis=1)
plt.bar(np.arange(7),correctly_predicted_frac )
plt.xlabel('True Label')
plt.ylabel('Correctly Classified Fraction')

In [ ]:
incorrectly_predicted_frac = 1 - np.diag(confusion_mtx) / np.sum(confusion_mtx, axis=1)
plt.bar(np.arange(7),incorrectly_predicted_frac )
plt.xlabel('True Label')
plt.ylabel('Incorrectly Classified Fraction')

In [ ]:
# Calculating model wise accuracy forensmble models and also calculating ensemble accuracy

from sklearn.metrics import accuracy_score

# makes prediction according to given models and given weights
def predict(models, data, weights=None):
    if weights is None:
        # default weights provide voting equality
        weights = [1 / (len(models))] * len(models)
    pred = np.zeros((data.shape[0], 7))
    for i, model in enumerate(models):
        pred += model.predict(data) * weights[i]
    return pred
    
# returns accuracy for given predictions
def evaluate(preds, weights=None):
    if weights is None:
        weights = [1 / len(preds)] * len(preds)
    y_pred = np.zeros((y_test.shape[0], 7))
    for i, pred in enumerate(preds):
        y_pred += pred * weights[i]
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)
    return accuracy_score(y_true, y_pred)

# load list of snapshots
models = se_callback.load_ensemble()
# precalculated predictions of all models
preds = []
# evaluate every model as single
for i, model in enumerate(models):
    pred = predict([model], x_test)
    preds.append(pred)
    score = evaluate([pred])
    print(f'model {i + 1}: accuracy = {score:.4f}')

# evaluate ensemble (with voting equality)
ensemble_score = evaluate(preds)
print(f'ensemble: accuracy = {ensemble_score:.4f}')

In [ ]:
# Improving ensemble accuracy

best_score = ensemble_score
best_weights = None
no_improvements = 0
while no_improvements < 5000: #patience
    
    # generate normalized weights
    new_weights = np.random.uniform(size=(len(models), ))
    new_weights /= new_weights.sum()
    
    # get the score without predicting again
    new_score = evaluate(preds, new_weights)
    
    # check (and save)
    if new_score > best_score:
        no_improvements = 0
        best_score = new_score
        best_weights = new_weights
        print(f'improvement: {best_score:.4f}')
    else:
        no_improvements += 1

print(f'best weights are {best_weights}')

In [ ]:
# Making predictions using best weights

pred = predict(models, x_test, best_weights)
pred_classes = np.argmax(pred, axis=1)
pred_classes_vector = to_categorical(pred_classes, num_classes = 7)

pred_val = predict(models, x_validate, best_weights)
pred_classes_val = np.argmax(pred_val, axis=1)
pred_classes_vector_val = to_categorical(pred_classes_val, num_classes = 7)

In [ ]:
classes = df['dx'].unique()
labeldict = {}
for num, name in enumerate(classes):
    labeldict[name] = num

In [ ]:
# Printing test and validation reports

from sklearn.metrics import classification_report
labels = labeldict.keys()
# Generate a classification report
validationreport = classification_report(y_validate, pred_classes_vector_val, target_names=list(labels))
testreport = classification_report(y_test, pred_classes_vector, target_names=list(labels))

print("Validation data Report")
print(validationreport)
print("Test data Report")
print(testreport)